In [13]:
import pandas as pd
import duckdb
import json
import requests as r
import zipfile
import io
import numpy as np
from analysis.ml.custom_transformers import *
from sklearn.pipeline import Pipeline

In [14]:
d = pd.read_parquet('/Users/louisspencer/LocalData/data/ohlcv_data/raw', engine='pyarrow')
d

,time_period_end,time_period_start,open,high,low,close,volume,trades,asset_id_base,asset_id_quote,exchange_id,symbol_id,date
0,2025-01-31 00:01:00,2025-01-31 00:00:00,0.000140,0.000140,0.000140,0.000140,550107.0,1,1000SATS,FDUSD,BINANCE,1000SATS_FDUSD_BINANCE,2025-01-31
1,2025-01-31 00:02:00,2025-01-31 00:01:00,0.000141,0.000141,0.000141,0.000141,14211.0,2,1000SATS,FDUSD,BINANCE,1000SATS_FDUSD_BINANCE,2025-01-31
2,2025-01-31 00:03:00,2025-01-31 00:02:00,0.000141,0.000141,0.000141,0.000141,7098.0,1,1000SATS,FDUSD,BINANCE,1000SATS_FDUSD_BINANCE,2025-01-31
3,2025-01-31 00:04:00,2025-01-31 00:03:00,0.000141,0.000141,0.000141,0.000141,3034459.0,3,1000SATS,FDUSD,BINANCE,1000SATS_FDUSD_BINANCE,2025-01-31
4,2025-01-31 00:05:00,2025-01-31 00:04:00,0.000141,0.000142,0.000141,0.000142,233628.0,10,1000SATS,FDUSD,BINANCE,1000SATS_FDUSD_BINANCE,2025-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
800635,2025-01-31 23:56:00,2025-01-31 23:55:00,0.413500,0.413500,0.413100,0.413400,7119.0,13,ZRX,USDT,BINANCE,ZRX_USDT_BINANCE,2025-01-31
800636,2025-01-31 23:57:00,2025-01-31 23:56:00,0.413600,0.413600,0.413600,0.413600,691.0,2,ZRX,USDT,BINANCE,ZRX_USDT_BINANCE,2025-01-31
800637,2025-01-31 23:58:00,2025-01-31 23:57:00,0.413600,0.413600,0.413600,0.413600,873.0,4,ZRX,USDT,BINANCE,ZRX_USDT_BINANCE,2025-01-31
800638,2025-01-31 23:59:00,2025-01-31 23:58:00,0.413800,0.413900,0.413800,0.413900,678.0,5,ZRX,USDT,BINANCE,ZRX_USDT_BINANCE,2025-01-31


In [15]:
d['symbol_id'].unique().tolist()

['1000SATS_FDUSD_BINANCE',
 '1000SATS_USDC_BINANCE',
 '1000SATS_USDT_BINANCE',
 '1INCH_USDT_BINANCE',
 '1MBABYDOGE_FDUSD_BINANCE',
 '1MBABYDOGE_USDT_BINANCE',
 'AAVE_FDUSD_BINANCE',
 'AAVE_USDC_BINANCE',
 'AAVE_USDT_BINANCE',
 'ACA_USDT_BINANCE',
 'ACE_USDT_BINANCE',
 'ACH_USDT_BINANCE',
 'ACM_USDT_BINANCE',
 'ADA_FDUSD_BINANCE',
 'ADA_USDC_BINANCE',
 'ADA_USDT_BINANCE',
 'ADX_USDT_BINANCE',
 'AERGO_USDT_BINANCE',
 'AEUR_USDT_BINANCE',
 'AEVO_FDUSD_BINANCE',
 'AEVO_USDT_BINANCE',
 'AGLD_USDT_BINANCE',
 'AI_FDUSD_BINANCE',
 'AI_USDT_BINANCE',
 'ALCX_USDT_BINANCE',
 'ALGO_USDC_BINANCE',
 'ALGO_USDT_BINANCE',
 'ALICE_USDT_BINANCE',
 'ALPACA_USDT_BINANCE',
 'ALPHA_USDT_BINANCE',
 'ALPINE_USDT_BINANCE',
 'ALT_FDUSD_BINANCE',
 'ALT_USDC_BINANCE',
 'ALT_USDT_BINANCE',
 'AMB_USDT_BINANCE',
 'AMP_USDT_BINANCE',
 'ANKR_USDT_BINANCE',
 'APE_USDT_BINANCE',
 'API3_USDT_BINANCE',
 'APT_FDUSD_BINANCE',
 'APT_USDC_BINANCE',
 'APT_USDT_BINANCE',
 'ARB_FDUSD_BINANCE',
 'ARB_TUSD_BINANCE',
 'ARB_USDC_BIN

In [4]:
from analysis.ml.custom_transformers import *

ModuleNotFoundError: No module named 'analysis'

In [7]:
feature_engineering_pipeline = Pipeline([
    ('time_features', TimeFeatures()),
    ('returns_features', ReturnsFeatures(
        window_sizes=[1, 7, 30, 180],
        lookback_windows=[30, 180])),
    ('risk_features',   RiskFeatures(
        windows=[1],
        lookback_windows=[30, 180])),
    ('trade_features',  TradeFeatures(
        windows=[1, 7],
        lookback_windows=[30, 180])),
    ('spot_fut_feats',  SpotFuturesInteractionFeatures(
        windows=[1, 7],
        lookback_windows=[30, 180])),
    ('rolling_z',       RollingZScoreScaler(window_sizes=[30])),
])


Calculating returns for 1000SATS_FDUSD_BINANCE with window size 1...
Calculating returns for 1000SATS_FDUSD_BINANCE with window size 7...


/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/.venv/src/trading-bot/analysis/ml/custom_transformers.py:186: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.ml_dataset.loc[filter, f'futures_returns_{window}'] = self.ml_dataset.loc[filter, 'close_futures'].pct_change(window).clip(-1, clip_upper_bound)
/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/.venv/src/trading-bot/analysis/ml/custom_transformers.py:186: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.ml_dataset.loc[filter, f'futures_returns_{window}'] = self.ml_dataset.loc[filter, 'close_futures'].pct_change(window).cli

Calculating returns for 1000SATS_FDUSD_BINANCE with window size 30...
Calculating returns for 1000SATS_FDUSD_BINANCE with window size 180...


/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/.venv/src/trading-bot/analysis/ml/custom_transformers.py:186: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.ml_dataset.loc[filter, f'futures_returns_{window}'] = self.ml_dataset.loc[filter, 'close_futures'].pct_change(window).clip(-1, clip_upper_bound)
/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/.venv/src/trading-bot/analysis/ml/custom_transformers.py:186: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.ml_dataset.loc[filter, f'futures_returns_{window}'] = self.ml_dataset.loc[filter, 'close_futures'].pct_change(window).cli

Calculating returns for 1000SATS_USDC_BINANCE with window size 1...
Calculating returns for 1000SATS_USDC_BINANCE with window size 7...
Calculating returns for 1000SATS_USDC_BINANCE with window size 30...
Calculating returns for 1000SATS_USDC_BINANCE with window size 180...


/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/.venv/src/trading-bot/analysis/ml/custom_transformers.py:186: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.ml_dataset.loc[filter, f'futures_returns_{window}'] = self.ml_dataset.loc[filter, 'close_futures'].pct_change(window).clip(-1, clip_upper_bound)
/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/.venv/src/trading-bot/analysis/ml/custom_transformers.py:186: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.ml_dataset.loc[filter, f'futures_returns_{window}'] = self.ml_dataset.loc[filter, 'close_futures'].pct_change(window).cli

Calculating returns for 1000SATS_USDT_BINANCE with window size 1...
Calculating returns for 1000SATS_USDT_BINANCE with window size 7...


KeyboardInterrupt: 

In [11]:
url = 'https://api.binance.com/api/v3/exchangeInfo'
response = r.get(url)
data = response.json()
df = pd.DataFrame(data['symbols'])
df.head()

,symbol,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,...,allowTrailingStop,cancelReplaceAllowed,amendAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions,permissionSets,defaultSelfTradePreventionMode,allowedSelfTradePreventionModes
0,ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_005, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH, DECR..."
1,LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_005, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH, DECR..."
2,BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_005, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH, DECR..."
3,NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_006, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH, DECR..."
4,QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, TRD_GRP_004, TRD_GRP_006, TRD_GRP_008,...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH, DECR..."


In [35]:
metadata = pd.read_json('/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/data/binance_metadata.json', lines = True)
metadata_symbols_id = (metadata['asset_id_base'] + '_' + metadata['asset_id_quote'] + '_' + metadata['exchange_id']).tolist()
len(metadata_symbols_id)

601

In [68]:
sub = df[
    (df['quoteAsset'].str.contains('USD')) & 
    (df['isSpotTradingAllowed']) &
    (df['isMarginTradingAllowed']) &
    ~(df['baseAsset'].str.contains('USD')) &
    ~(df['baseAsset'].str.contains('USTC'))
]
sub['symbol_id'] = sub['baseAsset'] + '_' + sub['quoteAsset'] + '_' + 'BINANCE'
sub_symbols_id = sub['symbol_id'].tolist()
sub.sort_values(by='symbol_id', inplace=True)
len(sub_symbols_id)

/var/folders/bn/k4t28qzx3qx1l712qxdnrxcm0000gn/T/ipykernel_46402/4164408775.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['symbol_id'] = sub['baseAsset'] + '_' + sub['quoteAsset'] + '_' + 'BINANCE'
/var/folders/bn/k4t28qzx3qx1l712qxdnrxcm0000gn/T/ipykernel_46402/4164408775.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.sort_values(by='symbol_id', inplace=True)


648

In [69]:
len(sub_symbols_id), len(metadata_symbols_id)

(648, 601)

In [70]:
symbol_id_set_sub = set(sub_symbols_id)
symbol_id_set_metadata = set(metadata_symbols_id)
diff = symbol_id_set_sub - symbol_id_set_metadata
print(f"Symbols in Binance not in metadata: {len(diff)}")

Symbols in Binance not in metadata: 172


In [71]:
new_metadata = metadata.copy()
for symbol_id in diff:
    base, quote, exchange = symbol_id.split('_')
    new_row = {
        'asset_id_base': base,
        'asset_id_quote': quote,
        'exchange_id': exchange,
        'candle_data_start': 1546300800000,  # 2019-01-01
        'candle_data_end': 1546300800000,  # 2019-01-01
        'trade_data_start': 1546300800000,  # 2019-01-01
        'trade_data_end': 1546300800000,  # 2019-01-01,
        'futures_trade_data_start': 1546300800000,  # 2019-01-01
        'futures_trade_data_end': 1546300800000,  # 2019-01-01
        'futures_candle_data_start': 1546300800000,  # 2019-01-01
        'futures_candle_data_end': 1546300800000,  # 2019-01-01,
        'funding_rate_data_start': 1546300800000,  # 2019-01-01
        'funding_rate_data_end': 1546300800000,  # 2019-01-01,
        'order_book_depth_data_start': 1546300800000,  # 2019-01-01
        'order_book_depth_data_end': 1546300800000,  # 2019-01-01,
        'futures_metrics_data_start': 1546300800000,  # 2019-01-01
        'futures_metrics_data_end': 1546300800000,  # 2019-01-01
    }
    new_metadata = pd.concat([new_metadata, pd.DataFrame([new_row])], ignore_index=True)

new_metadata.sort_values(by=['asset_id_base', 'asset_id_quote', 'exchange_id'], inplace=True)
new_metadata = new_metadata.reset_index(drop=True)
new_metadata


,asset_id_base,asset_id_quote,exchange_id,candle_data_start,candle_data_end,trade_data_start,trade_data_end,futures_trade_data_start,futures_trade_data_end,futures_candle_data_start,futures_candle_data_end,funding_rate_data_start,funding_rate_data_end,order_book_depth_data_start,order_book_depth_data_end,futures_metrics_data_start,futures_metrics_data_end
0,1000CAT,USDC,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
1,1000CAT,USDT,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
2,1000CHEEMS,USDC,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
3,1000CHEEMS,USDT,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
4,1000SATS,FDUSD,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,ZRO,FDUSD,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
769,ZRO,USDC,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
770,ZRO,USDT,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1746057600000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
771,ZRX,USDT,BINANCE,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1746057600000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000,1546300800000
